In [1]:
import torch

In [2]:
W = torch.randn((10, 1))
W

tensor([[ 1.4139],
        [-0.1277],
        [-1.0110],
        [ 1.4665],
        [ 0.1496],
        [-1.0984],
        [ 1.1004],
        [-0.0340],
        [-0.9898],
        [ 1.3108]])